In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder


import gc
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip3 install tensorflow_decision_forests

In [ ]:
import tensorflow_decision_forests as tfdf

In [ ]:
tfdf.keras.get_all_models()

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel()
?model

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
train.head()

In [ ]:
test_features = test.columns[1:]
train_features = train.columns[1:]
target = train['target'].values

In [ ]:
train_oof = np.zeros((train.shape[0],9))
test_preds = 0
train_oof.shape

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test[test_features])

In [ ]:
%%time

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_ds = train.loc[train_index]

    val_ds = train.loc[val_index]
    
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds[train_features], label="target")
    val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val_ds[train_features], label="target")
    model = tfdf.keras.GradientBoostedTreesModel(num_trees=700, subsample = 0.9, max_depth = 3)
    model.fit(x=train_ds)
    val_pred = model.predict(val_ds)
    train_oof[val_index] = val_pred
    #print("Fold AUC:", roc_auc_score(val_target, val_pred[:,1]))
    test_preds += model.predict(test_ds)/n_splits
    del train_ds, val_ds
    gc.collect()

    

In [ ]:
oho = OneHotEncoder()
log_loss(oho.fit_transform(target.reshape(-1,1)).toarray(), train_oof)

In [ ]:
sample_submission[sample_submission.columns[1:]] = test_preds
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)